In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install torch torchvision timm transformers scikit-learn tqdm numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

### original code 

In [45]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler, ConcatDataset, Subset
from torchvision import datasets, transforms, models
from collections import Counter
from PIL import Image
import warnings
import numpy as np
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import zipfile
warnings.filterwarnings("ignore")

# Set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("CUDA not available. Training on CPU.")

# Define paths for Kaggle environment
KAGGLE_INPUT_PATH = "/kaggle/input/plant-disease-detection"
KAGGLE_WORKING_PATH = "/kaggle/input/plant-disease-detection/disease-dataset - Copy"
MODEL_SAVE_PATH = "/kaggle/working/AI_Farmer_Models"
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
LOCAL_DATASET_PATH = KAGGLE_WORKING_PATH

class ImprovedCNNViTHybrid(nn.Module):
    def __init__(self, num_classes, pretrained=True):  # Fixed: __init__ instead of _init_
        super(ImprovedCNNViTHybrid, self).__init__()  # Fixed: __init__ instead of _init_
        self.backbone = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])
        
        self.feature_dim = 2048
        self.patch_size = 7
        self.num_patches = 49
        self.embedding_dim = 768
        
        self.feature_projection = nn.Sequential(
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Conv2d(self.feature_dim, self.embedding_dim, kernel_size=1),
            nn.BatchNorm2d(self.embedding_dim),
            nn.ReLU(inplace=True)
        )
        
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embedding_dim))
        self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, self.embedding_dim))
        self.dropout = nn.Dropout(0.3)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embedding_dim, nhead=8, dim_feedforward=2048, dropout=0.3, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=3)
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.embedding_dim),
            nn.Dropout(0.3),
            nn.Linear(self.embedding_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
    def forward(self, x):
        B = x.shape[0]
        features = self.backbone(x)
        features = self.feature_projection(features)
        features = features.flatten(2).transpose(1, 2)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        features = torch.cat([cls_tokens, features], dim=1)
        features = features + self.pos_embed
        features = self.dropout(features)
        encoded = self.transformer(features)
        cls_output = encoded[:, 0]
        return self.classifier(cls_output)

def get_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomRotation(degrees=10),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

def filter_valid_images(dataset, split_name):
    valid_indices = []
    for i in tqdm(range(len(dataset)), desc=f"Filtering valid images for {split_name}"):
        try:
            _ = dataset[i]
            valid_indices.append(i)
        except Exception as e:
            print(f"Skipping invalid image at index {i}: {e}")
    return Subset(dataset, valid_indices)

def load_datasets_improved(crop_name, base_path=LOCAL_DATASET_PATH):
    train_dir = os.path.join(base_path, 'Train', crop_name)
    val_dir = os.path.join(base_path, 'Validation', crop_name)
    test_dir = os.path.join(base_path, 'Test', crop_name)

    if not os.path.exists(train_dir):
        raise ValueError(f"Train directory for {crop_name} not found in {train_dir}")

    train_transform, val_transform = get_transforms()
    
    train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=val_transform) if os.path.exists(val_dir) else None
    test_dataset = datasets.ImageFolder(test_dir, transform=val_transform) if os.path.exists(test_dir) else None

    if val_dataset is None or test_dataset is None:
        raise ValueError(f"Validation or Test directory for {crop_name} not found")

    train_dataset = filter_valid_images(train_dataset, f"{crop_name} Train")
    val_dataset = filter_valid_images(val_dataset, f"{crop_name} Val")
    test_dataset = filter_valid_images(test_dataset, f"{crop_name} Test")

    class_counts = Counter([train_dataset.dataset.targets[i] for i in train_dataset.indices])
    print(f"Class distribution for {crop_name}:")
    for i, class_name in enumerate(train_dataset.dataset.classes):
        print(f"  {class_name}: {class_counts[i]} samples")

    num_classes = len(train_dataset.dataset.classes)
    print(f"Loaded {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test images for {crop_name}")
    
    return train_dataset, val_dataset, test_dataset, num_classes, train_dataset.dataset.classes

class OffsetLabelDataset:
    def __init__(self, dataset, offset):
        self.dataset = dataset
        self.offset = offset

    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label + self.offset

    def __len__(self):
        return len(self.dataset)

def get_weighted_sampler(combined_dataset, crops, class_offsets):
    # Fixed: Extract targets properly from combined dataset
    targets = []
    dataset_idx = 0
    
    for ds in combined_dataset.datasets:
        crop_name = crops[dataset_idx]
        offset = class_offsets[crop_name]
        
        # Get the actual targets from the subset
        for idx in range(len(ds)):
            _, label = ds[idx]  # This will give us the offset label
            targets.append(label)
        
        dataset_idx += 1
    
    class_counts = Counter(targets)
    num_classes = len(class_counts)
    
    # Calculate class weights
    class_weights = {}
    for class_idx, count in class_counts.items():
        if count > 0:
            class_weights[class_idx] = len(targets) / (num_classes * count)
    
    # Create sample weights
    sample_weights = [class_weights[target] for target in targets]
    
    return WeightedRandomSampler(weights=sample_weights, num_samples=len(targets), replacement=True)

def train_model_improved(model, train_loader, val_loader, model_name, num_epochs=40, lr=1e-4, device=device):
    model.to(device)
    
    # Fixed: Get targets from combined dataset properly
    val_targets = []
    for batch_inputs, batch_targets in val_loader:
        val_targets.extend(batch_targets.numpy())
    
    class_weights = compute_class_weight('balanced', classes=np.unique(val_targets), y=val_targets)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float).to(device))
    
    if isinstance(model, ImprovedCNNViTHybrid):
        lr = 1e-5
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    
    best_acc = 0.0
    best_loss = float('inf')
    patience = 25
    patience_counter = 0
    best_model_state = None
    
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in tqdm(range(num_epochs), desc=f"{model_name} Training"):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            if batch_idx % 50 == 0:
                print(f'Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()  # Fixed: += instead of =

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f'{model_name} Epoch {epoch+1}/{num_epochs}: Train Loss {train_loss:.4f}, Acc {train_acc:.4f} | '
              f'Val Loss {val_loss:.4f}, Acc {val_acc:.4f}')

        if val_acc > best_acc:
            best_acc = val_acc
            best_loss = val_loss
            best_model_state = model.state_dict()
            torch.save({
                'model_state_dict': best_model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'best_acc': best_acc,
            }, os.path.join(MODEL_SAVE_PATH, f'{model_name}_best_model.pth'))
            patience_counter = 0
        elif val_loss > best_loss:
            patience_counter += 1
        else:
            patience_counter = 0

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        scheduler.step(val_loss)

    # Load best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    return model, best_acc

def test_on_test_set(model, test_loader, model_name, class_names, device=device):
    model.eval()
    correct = 0
    total = 0
    class_correct = torch.zeros(len(class_names)).to(device)
    class_total = torch.zeros(len(class_names)).to(device)
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += (predicted[i] == label).item()
                class_total[label] += 1
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    acc = correct / total
    print(f"Test accuracy for {model_name}: {acc:.4f}")
    print("Per-class accuracy:")
    for i, (correct_count, total_count) in enumerate(zip(class_correct, class_total)):
        if total_count > 0 and i < len(class_names):
            print(f"  {class_names[i]}: {correct_count/total_count:.4f} ({correct_count}/{total_count})")
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names[:max(y_true)+1], digits=4))
    return acc, class_correct, class_total

def main_training():
    crops = ['pepper bell', 'Wheat', 'Rubber', 'Coffee', 'Banana', 'Cardamom', 'Coconut', 'Mango', 'Potato', 'Rice']
    batch_size = 16
    model_name = 'all_crops_hybrid'

    print(f"\n{'='*80}")
    print("LOADING COMBINED DATASETS FOR ALL CROPS")
    print('='*80)

    train_datasets = []
    val_datasets = []
    test_datasets = []
    global_classes = []
    total_num_classes = 0
    class_offsets = {}
    offset = 0

    total_train_samples = 0
    total_val_samples = 0
    total_test_samples = 0

    for crop in crops:
        train_ds, val_ds, test_ds, num_cls, cls_names = load_datasets_improved(crop)
        full_names = [crop + '_' + name for name in cls_names]
        global_classes.extend(full_names)
        class_offsets[crop] = offset
        train_datasets.append(OffsetLabelDataset(train_ds, offset))
        val_datasets.append(OffsetLabelDataset(val_ds, offset))
        test_datasets.append(OffsetLabelDataset(test_ds, offset))
        offset += num_cls
        total_num_classes += num_cls
        total_train_samples += len(train_ds)
        total_val_samples += len(val_ds)
        total_test_samples += len(test_ds)

    combined_train_dataset = ConcatDataset(train_datasets)
    combined_val_dataset = ConcatDataset(val_datasets)
    combined_test_dataset = ConcatDataset(test_datasets)

    print(f"\nCombined Dataset Summary:")
    print(f"  Training samples: {total_train_samples}")
    print(f"  Validation samples: {total_val_samples}")
    print(f"  Test samples: {total_test_samples}")
    print(f"  Number of classes: {total_num_classes}")
    print(f"  Classes: {global_classes}")

    print(f"\nAnalyzing class distribution and creating balanced sampler...")
    train_sampler = get_weighted_sampler(combined_train_dataset, crops, class_offsets)
    
    train_loader = DataLoader(combined_train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=2)
    val_loader = DataLoader(combined_val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    test_loader = DataLoader(combined_test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    print(f"\n{'-'*80}")
    print("TRAINING CNN-ViT HYBRID MODEL")
    print('-'*80)
    hybrid_model = ImprovedCNNViTHybrid(num_classes=total_num_classes)
    trained_model, best_acc = train_model_improved(
        hybrid_model, train_loader, val_loader, model_name, num_epochs=40, device=device
    )

    print(f"\n{'-'*80}")
    print("TESTING ON TEST SET")
    print('-'*80)
    test_acc, class_correct, class_total = test_on_test_set(trained_model, test_loader, model_name, global_classes, device)
    
    torch.save({
        'model_state_dict': trained_model.state_dict(),
        'model_type': 'hybrid',
        'num_classes': total_num_classes,
        'best_val_acc': best_acc,
        'test_acc': test_acc,
        'class_names': global_classes
    }, os.path.join(MODEL_SAVE_PATH, f'{model_name}_final_model.pth'))

    with open(os.path.join(MODEL_SAVE_PATH, f'{model_name}_classes.txt'), 'w') as f:
        f.write('\n'.join(global_classes))
    
    print(f"\n✅ Completed training: hybrid model with val_acc={best_acc:.4f}, test_acc={test_acc:.4f}")

if __name__ == "__main__":
    main_training()

GPU Available: Tesla T4
GPU Memory Available: 14.74 GB

LOADING COMBINED DATASETS FOR ALL CROPS


Filtering valid images for pepper bell Test: 100%|██████████| 505/505 [00:01<00:00, 375.38it/s]


Class distribution for pepper bell:
  Bacterial spot: 704 samples
  Healthy: 1049 samples
Loaded 1753 train, 217 val, 505 test images for pepper bell


Filtering valid images for Wheat Test: 100%|██████████| 500/500 [00:08<00:00, 59.40it/s]


Class distribution for Wheat:
  BlackPoint: 700 samples
  FusariumFootRot: 709 samples
  HealthyLeaf: 700 samples
  LeafBlight: 700 samples
  WheatBlast: 700 samples
Loaded 3509 train, 1000 val, 500 test images for Wheat


...

all_crops_hybrid Training:  98%|█████████▊| 39/40 [3:53:27<05:59, 359.78s/it]

all_crops_hybrid Epoch 39/40: Train Loss 0.0046, Acc 0.9964 | Val Loss 0.1391, Acc 0.9810
Batch 0/1375, Loss: 0.0017
Batch 50/1375, Loss: 0.0019
Batch 100/1375, Loss: 0.0014
Batch 150/1375, Loss: 0.0000
Batch 200/1375, Loss: 0.0000
Batch 250/1375, Loss: 0.0024
Batch 300/1375, Loss: 0.0000
Batch 350/1375, Loss: 0.0012
Batch 400/1375, Loss: 0.0000
Batch 450/1375, Loss: 0.0000
Batch 500/1375, Loss: 0.0006
Batch 550/1375, Loss: 0.0005
Batch 600/1375, Loss: 0.0000
Batch 650/1375, Loss: 0.0000
Batch 700/1375, Loss: 0.0000
Batch 750/1375, Loss: 0.0001
Batch 800/1375, Loss: 0.0000
Batch 850/1375, Loss: 0.0000
Batch 900/1375, Loss: 0.0007
Batch 950/1375, Loss: 0.0000
Batch 1000/1375, Loss: 0.0000
Batch 1050/1375, Loss: 0.0000
Batch 1100/1375, Loss: 0.0000
Batch 1150/1375, Loss: 0.0001
Batch 1200/1375, Loss: 0.0005
Batch 1250/1375, Loss: 0.0015
Batch 1300/1375, Loss: 0.0000
Batch 1350/1375, Loss: 0.0000


all_crops_hybrid Training: 100%|██████████| 40/40 [3:59:26<00:00, 359.15s/it]

all_crops_hybrid Epoch 40/40: Train Loss 0.0050, Acc 0.9961 | Val Loss 0.1378, Acc 0.9813

--------------------------------------------------------------------------------
TESTING ON TEST SET
--------------------------------------------------------------------------------


Test accuracy for all_crops_hybrid: 0.9535
Per-class accuracy:
  pepper bell_Bacterial spot: 1.0000 (205.0/205.0)
  pepper bell_Healthy: 1.0000 (300.0/300.0)
  Wheat_BlackPoint: 1.0000 (100.0/100.0)
  Wheat_FusariumFootRot: 1.0000 (100.0/100.0)
  Wheat_HealthyLeaf: 1.0000 (100.0/100.0)
  Wheat_LeafBlight: 1.0000 (100.0/100.0)
  Wheat_WheatBlast: 1.0000 (100.0/100.0)
  Rubber_Anthracnose: 1.0000 (62.0/62.0)
  Rubber_Dry leaf: 1.0000 (90.0/90.0)
  Rubber_Healthy: 1.0000 (101.0/101.0)
  Rubber_Leaf Spot: 1.0000 (95.0/95.0)
  Coffee_Healthy: 0.9873 (234.0/237.0)
  Coffee_Phoma: 0.6233 (139.0/223.0)
  Coffee_leaf rust: 0.9420 (195.0/207.0)
  Banana_Cordana: 1.0000 (80.0/80.0)
  Banana_Healthy: 1.0000 (80.0/80.0)
  Banana_Sigatoka: 0.8875 (71.0/80.0)
  Banana_pastalotiopsis: 0.9500 (76.0/80.0)
  Cardamom_Blight: 1.0000 (28.0/28.0)
  Cardamom_Healthy: 1.0000 (79.0/79.0)
  Cardamom_Phylosticta: 1.0000 (67.0/67.0)
  Coconut_Caterpillar: 1.0000 (202.0/202.0)
  Coconut_Drying of leaves: 1.0000 (9

In [46]:
!ls -l /kaggle/working/AI_Farmer_Models

total 658380
-rw-r--r-- 1 root root 505499806 Sep 11 02:56 all_crops_hybrid_best_model.pth
-rw-r--r-- 1 root root       747 Sep 11 03:09 all_crops_hybrid_classes.txt
-rw-r--r-- 1 root root 168673193 Sep 11 03:09 all_crops_hybrid_final_model.pth


In [48]:
ritam = '/kaggle/working/AI_Farmer_Models'

In [49]:
ritam

'/kaggle/working/AI_Farmer_Models'

In [50]:
checkpoint = torch.load("/kaggle/working/AI_Farmer_Models/all_crops_hybrid_final_model.pth")
print(checkpoint.keys())  # Should include 'model_state_dict', 'model_type', 'num_classes', 'best_val_acc', 'test_acc', 'class_names'

dict_keys(['model_state_dict', 'model_type', 'num_classes', 'best_val_acc', 'test_acc', 'class_names'])


In [51]:
with open("/kaggle/working/AI_Farmer_Models/all_crops_hybrid_classes.txt", 'r') as f:
    class_names = f.read().splitlines()
print(class_names)

['pepper bell_Bacterial spot', 'pepper bell_Healthy', 'Wheat_BlackPoint', 'Wheat_FusariumFootRot', 'Wheat_HealthyLeaf', 'Wheat_LeafBlight', 'Wheat_WheatBlast', 'Rubber_Anthracnose', 'Rubber_Dry leaf', 'Rubber_Healthy', 'Rubber_Leaf Spot', 'Coffee_Healthy', 'Coffee_Phoma', 'Coffee_leaf rust', 'Banana_Cordana', 'Banana_Healthy', 'Banana_Sigatoka', 'Banana_pastalotiopsis', 'Cardamom_Blight', 'Cardamom_Healthy', 'Cardamom_Phylosticta', 'Coconut_Caterpillar', 'Coconut_Drying of leaves', 'Coconut_Flaccidity', 'Coconut_Healthy leaves', 'Coconut_Yellowing', 'Coconut_leaflet', 'Mango_Alternaria', 'Mango_Anthracnose', 'Mango_Black Mould rot', 'Mango_Healthy', 'Mango_Stem and root', 'Potato_Potato___Early_blight', 'Potato_Potato___Late_blight', 'Potato_Potato___healthy', 'Rice_Bacterial leaf Blight', 'Rice_Healthy leaf', 'Rice_Rice', 'Rice_Rice Blast', 'Rice_Tungro']


### Testing via image url

In [69]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from PIL import Image
import os
import requests
from io import BytesIO
import base64

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model architecture
class ImprovedCNNViTHybrid(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ImprovedCNNViTHybrid, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])
        
        self.feature_dim = 2048
        self.patch_size = 7
        self.num_patches = 49
        self.embedding_dim = 768
        
        self.feature_projection = nn.Sequential(
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Conv2d(self.feature_dim, self.embedding_dim, kernel_size=1),
            nn.BatchNorm2d(self.embedding_dim),
            nn.ReLU(inplace=True)
        )
        
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embedding_dim))
        self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, self.embedding_dim))
        self.dropout = nn.Dropout(0.3)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embedding_dim, nhead=8, dim_feedforward=2048, dropout=0.3, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=3)
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.embedding_dim),
            nn.Dropout(0.3),
            nn.Linear(self.embedding_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
    def forward(self, x):
        B = x.shape[0]
        features = self.backbone(x)
        features = self.feature_projection(features)
        features = features.flatten(2).transpose(1, 2)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        features = torch.cat([cls_tokens, features], dim=1)
        features = features + self.pos_embed
        features = self.dropout(features)
        encoded = self.transformer(features)
        cls_output = encoded[:, 0]
        return self.classifier(cls_output)

# Load the trained model
def load_trained_model(model_path, device=device):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file {model_path} not found.")
    checkpoint = torch.load(model_path, map_location=device)
    num_classes = checkpoint.get('num_classes', 40)  # Default to 40 if not found
    model = ImprovedCNNViTHybrid(num_classes=num_classes, pretrained=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    # Extract class names from checkpoint, fallback to file if not present
    class_names = checkpoint.get('class_names', None)
    if class_names is None:
        class_names_file = model_path.replace('_final_model.pth', '_classes.txt')
        if os.path.exists(class_names_file):
            with open(class_names_file, 'r') as f:
                class_names = f.read().splitlines()
        else:
            raise FileNotFoundError(f"Class names file {class_names_file} not found.")
    return model, class_names

# Define the same transformations used during training
def get_test_transform():
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.Grayscale(num_output_channels=3),  # Adjust based on your dataset (remove if RGB)
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Download image from URL and test the model (updated for base64)
def test_on_image_url(model, class_names, image_url, device=device, threshold=0.8):
    # Handle base64 URL
    if image_url.startswith('data:image'):
        try:
            header, encoded = image_url.split(',', 1)
            img_data = base64.b64decode(encoded)
            img = Image.open(BytesIO(img_data)).convert('RGB')
        except Exception as e:
            print(f"Error decoding base64 image: {e}")
            return "Invalid image. Contact local Kisan Officer at 1800-180-1551.", None
    else:
        # Download from regular URL
        try:
            response = requests.get(image_url, timeout=10)
            response.raise_for_status()  # Check for HTTP errors
            img = Image.open(BytesIO(response.content)).convert('RGB')
        except Exception as e:
            print(f"Error downloading or loading image from {image_url}: {e}")
            return "Invalid image. Contact local Kisan Officer at 1800-180-1551.", None

    # Preprocess the image
    transform = get_test_transform()
    img = transform(img).unsqueeze(0).to(device)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(img)
        prob = torch.softmax(output, dim=1)[0]
        pred_idx = prob.argmax().item()
        confidence = prob[pred_idx].item()

    # Map prediction to class name
    if pred_idx >= len(class_names):
        print(f"Warning: Predicted index {pred_idx} exceeds number of classes {len(class_names)}")
        return "Invalid prediction. Contact local Kisan Officer at 1800-180-1551.", None

    predicted_class = class_names[pred_idx]

    # Determine if it's a disease (assuming class names end with '_Healthy' or disease names)
    is_disease = not predicted_class.endswith('_Healthy')
    disease_status = "Disease detected" if is_disease else "No disease detected (Healthy)"

    # Decision based on confidence
    if confidence < threshold:
        print(f"Prediction for {image_url}: Not confident (Confidence: {confidence:.4f})")
        return f"Not confident in prediction. {disease_status}. Contact local Kisan Officer at 1800-180-1551.", confidence
    else:
        print(f"Prediction for {image_url}: {predicted_class} (Confidence: {confidence:.4f})")
        return f"{predicted_class} - {disease_status} (Confidence: {confidence:.4f})", confidence

# Main execution
if __name__ == "__main__":
    # Paths to the model and class names file
    model_path = "/kaggle/working/AI_Farmer_Models/all_crops_hybrid_final_model.pth"
    class_names_file = "/kaggle/working/AI_Farmer_Models/all_crops_hybrid_classes.txt"

    # Load the model and class names
    model, class_names = load_trained_model(model_path)
    # Optionally override with file content if needed
    with open(class_names_file, 'r') as f:
        class_names = f.read().splitlines()

    # Provide the URL of the image to test
    image_url = "https://apps.lucidcentral.org/pppw_v10/images/entities/mango_bacterial_black_spot_213/mangobactspot1.jpg"

    # Test the model on the image from URL
    prediction, confidence = test_on_image_url(model, class_names, image_url)
    print(f"Final Prediction: {prediction}, Confidence: {confidence:.4f}")

Prediction for https://apps.lucidcentral.org/pppw_v10/images/entities/mango_bacterial_black_spot_213/mangobactspot1.jpg: Rubber_Leaf Spot (Confidence: 0.8608)
Final Prediction: Rubber_Leaf Spot - Disease detected (Confidence: 0.8608), Confidence: 0.8608


### Testing via downloaded images

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from PIL import Image
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model architecture
class ImprovedCNNViTHybrid(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ImprovedCNNViTHybrid, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])
        
        self.feature_dim = 2048
        self.patch_size = 7
        self.num_patches = 49
        self.embedding_dim = 768
        
        self.feature_projection = nn.Sequential(
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Conv2d(self.feature_dim, self.embedding_dim, kernel_size=1),
            nn.BatchNorm2d(self.embedding_dim),
            nn.ReLU(inplace=True)
        )
        
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embedding_dim))
        self.pos_embed = nn.Parameter(torch.randn(1, self.num_patches + 1, self.embedding_dim))
        self.dropout = nn.Dropout(0.3)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embedding_dim, nhead=8, dim_feedforward=2048, dropout=0.3, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=3)
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.embedding_dim),
            nn.Dropout(0.3),
            nn.Linear(self.embedding_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        
    def forward(self, x):
        B = x.shape[0]
        features = self.backbone(x)
        features = self.feature_projection(features)
        features = features.flatten(2).transpose(1, 2)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        features = torch.cat([cls_tokens, features], dim=1)
        features = features + self.pos_embed
        features = self.dropout(features)
        encoded = self.transformer(features)
        cls_output = encoded[:, 0]
        return self.classifier(cls_output)

# Load the trained model
def load_trained_model(model_path, device=device):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file {model_path} not found.")
    checkpoint = torch.load(model_path, map_location=device)
    num_classes = checkpoint.get('num_classes', 40)  # Default to 40 if not found
    model = ImprovedCNNViTHybrid(num_classes=num_classes, pretrained=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    # Extract class names from checkpoint, fallback to file if not present
    class_names = checkpoint.get('class_names', None)
    if class_names is None:
        class_names_file = model_path.replace('_final_model.pth', '_classes.txt')
        if os.path.exists(class_names_file):
            with open(class_names_file, 'r') as f:
                class_names = f.read().splitlines()
        else:
            raise FileNotFoundError(f"Class names file {class_names_file} not found.")
    return model, class_names

# Define the same transformations used during training
def get_test_transform():
    return transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.Grayscale(num_output_channels=3),  # Adjust based on your dataset (remove if RGB)
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Test the model on an image from the input directory
def test_on_image(model, class_names, image_path, device=device, threshold=0.8):
    # Load and preprocess the image
    try:
        img = Image.open(image_path).convert('RGB')
        print(f"Loaded image from {image_path}, size: {img.size}")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return "Invalid image. Contact local Kisan Officer at 1800-180-1551.", None

    transform = get_test_transform()
    img = transform(img).unsqueeze(0).to(device)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(img)
        prob = torch.softmax(output, dim=1)[0]
        pred_idx = prob.argmax().item()
        confidence = prob[pred_idx].item()
        print(f"Raw probabilities: {prob}")
        print(f"Predicted index: {pred_idx}, Confidence: {confidence}")

    # Map prediction to class name
    if pred_idx >= len(class_names):
        print(f"Warning: Predicted index {pred_idx} exceeds number of classes {len(class_names)}")
        return "Invalid prediction. Contact local Kisan Officer at 1800-180-1551.", None

    predicted_class = class_names[pred_idx]

    # Determine if it's a disease (assuming class names end with '_Healthy' or disease names)
    is_disease = not predicted_class.endswith('_Healthy')
    disease_status = "Disease detected" if is_disease else "No disease detected (Healthy)"

    # Decision based on confidence
    if confidence < threshold:
        print(f"Prediction for {image_path}: Not confident (Confidence: {confidence:.4f})")
        return f"Not confident in prediction. {disease_status}. Contact local Kisan Officer at 1800-180-1551.", confidence
    else:
        print(f"Prediction for {image_path}: {predicted_class} (Confidence: {confidence:.4f})")
        return f"{predicted_class} - {disease_status} (Confidence: {confidence:.4f})", confidence

# Main execution
if __name__ == "__main__":
    # Paths to the model and class names file
    model_path = "/kaggle/working/AI_Farmer_Models/all_crops_hybrid_final_model.pth"
    class_names_file = "/kaggle/working/AI_Farmer_Models/all_crops_hybrid_classes.txt"

    # Load the model and class names
    model, class_names = load_trained_model(model_path)
    with open(class_names_file, 'r') as f:
        class_names = f.read().splitlines()
    print(f"Loaded {len(class_names)} classes: {class_names[:5]}...")

    # Path to the uploaded image in the input directory
    image_path = "/kaggle/input/testimgs/Test image/1.jpeg"  # Adjust this path based on your upload

    # Test the model on the image
    prediction, confidence = test_on_image(model, class_names, image_path)
    if confidence is not None:
        print(f"Final Prediction: {prediction}, Confidence: {confidence:.4f}")
    else:
        print(f"Final Prediction: {prediction}")

FileNotFoundError: Model file /kaggle/working/AI_Farmer_Models/all_crops_hybrid_final_model.pth not found.